In [ ]:
from tvsclib.strict_system import StrictSystem
from tvsclib.stage import Stage

from tvsclib.mixed_system import MixedSystem
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as linalg
import tvsclib.utils as utils
import tvsclib.math as math

from tvsclib.identification import identify
import scipy.linalg as linalg

In [ ]:
dims_in = np.ones(4,dtype=int)*5
dims_out = np.ones(4,dtype=int)*5
matrix  = np.random.rand(dims_out.sum(),dims_out.sum())
system,sigmas = identify(matrix,dims_in,dims_out,compute_sigmas=True)
utils.show_system(system)

# Cut states according to one state 

In [ ]:
def approx_state(system,sigmas,k,eps):
    sys_copy = system.copy()
    #causal
    d = np.count_nonzero(sigmas[0][k-1]>eps)
    print(d)
    sigmas_cut= (sigmas[0][k-1][d:])
    sys_copy.causal_system.stages[k-1].B_matrix=sys_copy.causal_system.stages[k-1].B_matrix[:d,:]
    sys_copy.causal_system.stages[k-1].A_matrix=sys_copy.causal_system.stages[k-1].A_matrix[:d,:]
    sys_copy.causal_system.stages[k].C_matrix=sys_copy.causal_system.stages[k].C_matrix[:,:d]
    sys_copy.causal_system.stages[k].A_matrix=sys_copy.causal_system.stages[k].A_matrix[:,:d]
    
    #anticausal
    d = np.count_nonzero(sigmas[1][k-1]>eps)
    print(d)
    sigmas_cut_anti= (sigmas[1][k-1][d:])
    sys_copy.anticausal_system.stages[k].B_matrix=sys_copy.anticausal_system.stages[k].B_matrix[:d,:]
    sys_copy.anticausal_system.stages[k].A_matrix=sys_copy.anticausal_system.stages[k].A_matrix[:d,:]
    sys_copy.anticausal_system.stages[k-1].C_matrix=sys_copy.anticausal_system.stages[k-1].C_matrix[:,:d]
    sys_copy.anticausal_system.stages[k-1].A_matrix=sys_copy.anticausal_system.stages[k-1].A_matrix[:,:d]

    return sys_copy,(sigmas_cut,sigmas_cut_anti)

In [ ]:
sys_aprox,sigmas_cut = approx_state(system,sigmas,1,1.1)
utils.check_dims(sys_aprox)
utils.show_system(sys_aprox)
print(sys_aprox)
sigmas_cut

### Compare the approxiamtion error with cut singular values

For one state we can comput the appeoxiamtion error usig the truncated singular values

In [ ]:
print("Frobenius:")
print(np.linalg.norm(matrix-sys_aprox.to_matrix()))
print(np.sqrt(sum([np.sum(s**2) for s in sigmas_cut])))

In [ ]:
print("Spectral")
print(np.linalg.norm(matrix-sys_aprox.to_matrix(),2))
print(max([np.max(s) for s in sigmas_cut]))

### Compute Approxiamtion and upper bound

$$\|M -\hat{T}\| \leq \sum_i \|M -\hat{T}^{(i)}\|$$

In [ ]:
#eps = 10#
eps = 1.0
#eps = 0.5
sigmas_cut = []
sys_aprox = system
for i in range(len(system.dims_in)-1):
    print("i:",i)
    sys_aprox,sigmas_cuta = approx_state(sys_aprox,sigmas,i+1,eps)
    sigmas_cut.append(sigmas_cuta)
utils.check_dims(sys_aprox)
utils.show_system(sys_aprox)
print(sys_aprox)
sigmas_cuta
sigmas_cutb

In [ ]:
print("Frobenius:")
err = np.linalg.norm(matrix-sys_aprox.to_matrix())
print(err)
bound = np.sum([np.sqrt(sum([np.sum(s**2) for s in sigs])) for sigs in sigmas_cut])
print(bound)
err<=bound

In [ ]:
print("Spectral")
err = np.linalg.norm(matrix-sys_aprox.to_matrix(),2)
print(err)
bound = np.sum([max([np.max(s) for s in sigs]) for sigs in sigmas_cut])
print(bound)
err<=bound

## Test parts of the proof:

$\|\breve{\Delta}_k\| \leq \|\Delta_k\|$

In [ ]:
dims_in = np.ones(6,dtype=int)*5
dims_out = np.ones(6,dtype=int)*5
matrix  = np.random.rand(dims_out.sum(),dims_out.sum())
system,sigmas = identify(matrix,dims_in,dims_out,compute_sigmas=True)
utils.show_system(system)
eps = 2

In [ ]:
Delta1 = system.to_matrix()-approx_state(system,sigmas,1,eps)[0].to_matrix()
plt.matshow(Delta1)
#Delta1breve = 

In [ ]:
sys_aprox_part = system
for i in range(1,len(system.dims_in)-1):
    print("i:",i)
    sys_aprox_part,sigmas_cuta = approx_state(sys_aprox_part,sigmas,i+1,eps)
Delta1breve = sys_aprox_part.to_matrix()-approx_state(sys_aprox_part,sigmas,1,eps)[0].to_matrix()
plt.matshow(Delta1breve)

In [ ]:
np.linalg.norm(Delta1breve)<=np.linalg.norm(Delta1)

In [ ]:
np.linalg.norm(Delta1breve,2)<=np.linalg.norm(Delta1,2)

Now test decomposition of sum

$$
\|\Delta_k u\|_2^2
	  =
	    \|\breve{\Delta}_k u\|_2^2
	  + \dots +
	  \Big\|
	  \mathcal{O}_{k+2[2]}A_{k+2[21]}A_{k[12]}
	  \mathcal{R}_{k[2]} u
	  \Big\|_2^2
	  + \Big\|
	  \mathcal{O}_{k+1[2]}A_{k[22]}
	  \mathcal{R}_{k[2]} u
	  \Big\|_2^2
$$

In [ ]:
D = Delta1[5:,:5]
Dbreve = Delta1breve[5:,:5]

In [ ]:
ds = [np.count_nonzero(sig>eps) for sig in sigmas[0]]
R_cut = system.causal_system.stages[0].B_matrix[ds[0]:,:]

u = np.random.rand(R_cut.shape[1],1)

A = [stage.A_matrix for stage in system.causal_system.stages]
mats = [system.causal_system.observability_matrix(2)[:,ds[1]:]@A[1][ds[1]:,ds[0]:]@R_cut]

for i in range(2,len(A)-1):
    mat = A[1][:ds[1],ds[0]:]@R_cut
    for j in range(2,i):
        mat = A[j][:ds[j],:ds[j-1]]@mat
    mat = system.causal_system.observability_matrix(i+1)[:,ds[i]:]@ A[i][ds[i]:,:ds[i-1]] @mat
    mats.append(mat)


norm_ref = np.linalg.norm(D@u)**2
norm = np.linalg.norm(Dbreve@u)**2+sum([np.linalg.norm(mat@u)**2 for mat in mats])
print(norm_ref)
print(norm)